# Assignment 1: Algorithmic Fairness Definitions

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


This component of the assignment derives in part, with thanks and permission, from an [assignment](https://web.stanford.edu/class/cs182/assignments/AlgorithmicDecisionMaking.zip) in Stanford's CS182: Ethics, Public Policy, and Technological Change. Their assignment, in turn, is based on the journalistic organization ProPublica's [analysis](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing) of a criminal risk prediction algorithm which we discussed in the algorithmic fairness lecture. Here, you will be assessing how a classifier designed to predict recidivism -- that is, whether someone will commit a crime in the future -- performs in terms of algorithmic fairness metrics.

## Problem 1: Loading the data (5 points)

We have split the data for you into a train set (`recidivism-training-data.csv`) and test set (`recidivism-testing-data.csv`). You will be training the model on the train set and evaluating model predictions on the test set.

1a. Read in the train and test sets. Display the first 10 rows of the data. (4 points)

In [2]:
training_data = pd.read_csv("recidivism-testing-data.csv")
training_data.head(10)

,Juvenile felony count = 0,Juvenile felony count = 1,Juvenile felony count = 2,Juvenile felony count >= 3,Juvenile misdemeanor count = 0,Juvenile misdemeanor count = 1,Juvenile misdemeanor count = 2,Juvenile misdemeanor count >= 3,Juvenile other offense count = 0,Juvenile other offense count = 1,...,Age > 45,Gender = Female,Gender = Male,Race = Other,Race = Asian,Race = Native American,Race = Caucasian,Race = Hispanic,Race = African American,recidivism_outcome
0,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
1,1,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
2,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
3,1,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,1
4,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,1
5,1,0,0,0,1,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
6,1,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,1
7,1,0,0,0,1,0,0,0,1,0,...,1,1,0,0,0,0,0,0,1,1
8,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
9,1,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,1


In [3]:
testing_data = pd.read_csv("recidivism-testing-data.csv")
testing_data.head(10)

,Juvenile felony count = 0,Juvenile felony count = 1,Juvenile felony count = 2,Juvenile felony count >= 3,Juvenile misdemeanor count = 0,Juvenile misdemeanor count = 1,Juvenile misdemeanor count = 2,Juvenile misdemeanor count >= 3,Juvenile other offense count = 0,Juvenile other offense count = 1,...,Age > 45,Gender = Female,Gender = Male,Race = Other,Race = Asian,Race = Native American,Race = Caucasian,Race = Hispanic,Race = African American,recidivism_outcome
0,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
1,1,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
2,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
3,1,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,1
4,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,1
5,1,0,0,0,1,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
6,1,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,1
7,1,0,0,0,1,0,0,0,1,0,...,1,1,0,0,0,0,0,0,1,1
8,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
9,1,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,1


1b. Read the data documentation in the `Algorithmic Fairness Data Documentation` file. What are the possible values of the `Race` variable in the dataset? (1 point)

The specificied race can be one of the following:
- Asian
- Native American
- Caucasian (or white)
- Hispanic
- African-American (or “Black”)
- Other (none of the races above)

This means that there is a one-hot vector for the person's race, and it is the value 1 for their one selected race, and 0 for the other boxes.

## Problem 2: Predicting Recidivism with a Full Logistic Regression (20 points)

Now you will use the train set to train a logistic regression model using `sklearn.linear_model.LogisticRegression`. To train a logistic regression model on features X to predict outcome y, you can use the command:

`LogisticRegression(penalty='none').fit(X, y)`

You will have to replace X and y with the data you actually want to use. [Here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) is the documentation on logistic regression. Use the "recidivism_outcome" column as the variable you are trying to predict (y).

Then, we will ask you to report model performance metrics. Use the test set to compute those quantities. You could take a look at helpful functions from the [scikit-learn metrics module](https://scikit-learn.org/stable/modules/model_evaluation.html), imported at the beginning of the assignment as `metrics`.

2a. Train a logistic regression model using all features (except `recidivism_outcome`) as input features X. (5 points)

In [5]:
train_input_features = training_data.drop(columns = "recidivism_outcome")
train_output_feature = training_data["recidivism_outcome"]

logistic_regression_model = LogisticRegression(penalty='none', max_iter=1000000).fit(train_input_features, train_output_feature)

2b. Predict recidivism for the test set. Display the first 10 predictions. (1 point)

In [6]:
test_input_features = testing_data.drop(columns=["recidivism_outcome"])
test_predictions = logistic_regression_model.predict(test_input_features)
test_actual = testing_data["recidivism_outcome"]

print("Test preditctions: ")
print(test_predictions[:10])
print("Test actual: ")
print(testing_data["recidivism_outcome"][:10])

Test preditctions: 
[1 0 1 1 0 0 0 0 0 1]
Test actual: 
0    0
1    0
2    0
3    1
4    1
5    0
6    1
7    1
8    0
9    1
Name: recidivism_outcome, dtype: int64


2c. Report your model's AUC (i) for all defendants, (ii) for white defendants, and (iii) for Black defendants. Report the values with 4 decimal points. (1 point)

In [7]:
def get_accuracy(predictions, actual):
    return round((predictions == actual).mean(), 4)

def print_accuracy(data, predictions, actual, label):
    acc = get_accuracy(predictions, actual)
    print(f"{label} defendants accuracy: {acc}")

all_defendants_predictions = logistic_regression_model.predict(testing_data.drop(columns=['recidivism_outcome']))
black_defendants_predictions = logistic_regression_model.predict(testing_data.loc[testing_data["Race = African American"] == 1].drop(columns=['recidivism_outcome']))
white_defendants_predictions = logistic_regression_model.predict(testing_data.loc[testing_data["Race = Caucasian"] == 1].drop(columns=['recidivism_outcome']))

print_accuracy(testing_data, all_defendants_predictions, testing_data["recidivism_outcome"], "All")
print_accuracy(testing_data.loc[testing_data["Race = African American"] == 1], black_defendants_predictions, testing_data.loc[testing_data["Race = African American"] == 1]["recidivism_outcome"], "Black")
print_accuracy(testing_data.loc[testing_data["Race = Caucasian"] == 1], white_defendants_predictions, testing_data.loc[testing_data["Race = Caucasian"] == 1]["recidivism_outcome"], "White")



All defendants accuracy: 0.6774
Black defendants accuracy: 0.6625
White defendants accuracy: 0.6895


2d. Report your model's false positive rate (i) for all defendants, (ii) for white defendants, and (iii) for Black defendants. Report the values with 4 decimal points. (1 point)

In [8]:
def get_false_positive_rate(predictions, actual):
    false_positives = ((predictions == 1) & (actual == 0)).sum()
    total_negatives = (actual == 0).sum()
    return round(false_positives / total_negatives, 4)

all_defendants_predictions = logistic_regression_model.predict(testing_data.drop(columns=['recidivism_outcome']))
black_defendants_predictions = logistic_regression_model.predict(testing_data.loc[testing_data["Race = African American"] == 1].drop(columns=['recidivism_outcome']))
white_defendants_predictions = logistic_regression_model.predict(testing_data.loc[testing_data["Race = Caucasian"] == 1].drop(columns=['recidivism_outcome']))

fpr_all = get_false_positive_rate(all_defendants_predictions, testing_data["recidivism_outcome"])
fpr_black = get_false_positive_rate(black_defendants_predictions, testing_data.loc[testing_data["Race = African American"] == 1]["recidivism_outcome"])
fpr_white = get_false_positive_rate(white_defendants_predictions, testing_data.loc[testing_data["Race = Caucasian"] == 1]["recidivism_outcome"])

print("All defendants false positive rate: " + str(fpr_all))
print("Black defendants false positive rate: " + str(fpr_black))
print("White defendants false positive rate: " + str(fpr_white))

All defendants false positive rate: 0.2563
Black defendants false positive rate: 0.3578
White defendants false positive rate: 0.183


2e. Report your model's false negative rate (i) for all defendants, (ii) for white defendants, and (iii) for Black defendants. Report the values with 4 decimal points. (1 point)

In [9]:
def get_false_negative_rate(predictions, actual):
    false_negatives = ((predictions == 0) & (actual == 1)).sum()
    total_positives = (actual == 1).sum()
    return round(false_negatives / total_positives, 4)

fnr_all = get_false_negative_rate(all_defendants_predictions, testing_data["recidivism_outcome"])
fnr_black = get_false_negative_rate(black_defendants_predictions, testing_data.loc[testing_data["Race = African American"] == 1]["recidivism_outcome"])
fnr_white = get_false_negative_rate(white_defendants_predictions, testing_data.loc[testing_data["Race = Caucasian"] == 1]["recidivism_outcome"])

print("All defendants false negative rate:", fnr_all)
print("Black defendants false negative rate:", fnr_black)
print("White defendants false negative rate:", fnr_white)


All defendants false negative rate: 0.4048
Black defendants false negative rate: 0.3181
White defendants false negative rate: 0.5124


2f. Report the fraction of defendants classified positive by your model (i) for all defendants, (ii) for white defendants, and (iii) for Black defendants. Report the values with 4 decimal points. (1 point)

In [10]:
def get_predicted_positive(predictions, actual):
    tp = ((predictions == 1) & (actual == 1)).sum()
    fp = ((predictions == 1) & (actual == 0)).sum()
    tn = ((predictions == 0) & (actual == 0)).sum()
    fn = ((predictions == 0) & (actual == 1)).sum()

    predicted_positive_rate = (tp + fp) / (tp + fn + fp + tn)
    return round(predicted_positive_rate, 4)

predicted_positive_all = get_predicted_positive(all_defendants_predictions, testing_data["recidivism_outcome"])
predicted_positive_black = get_predicted_positive(black_defendants_predictions, testing_data.loc[testing_data["Race = African American"] == 1]["recidivism_outcome"])
predicted_positive_white = get_predicted_positive(white_defendants_predictions, testing_data.loc[testing_data["Race = Caucasian"] == 1]["recidivism_outcome"])

print("All defendants predicted positive rate:", predicted_positive_all)
print("Black defendants predicted positive rate:", predicted_positive_black)
print("White defendants predicted positive rate:", predicted_positive_white)

All defendants predicted positive rate: 0.4076
Black defendants predicted positive rate: 0.5233
White defendants predicted positive rate: 0.301


2g. In at least 5 sentences, describe what you observe, and any algorithmic fairness concerns it raises, making reference to specific algorithmic fairness concepts discussed in class and using quantitative evidence from 2c-f. Do you believe this algorithm is fair enough to be deployed in practice? Why or why not? (10 points)

The above trained algorithm raises a few algorithmic fairness concerns. 

First, it does not satisfy the predictive equality (also known as equalized odds) measurment, since Black defendants who do not reoffend are more likely than white defendant that do not reoffend to be predicted to be someone who will reoffend. This is evident in 2.d, where we show that this model has a false positive rate of 0.3578 for Black defendants and a false positive rate of 0.183 for white defendants, meaning that about twice as many Black defendants are falsely classified as someone who will reoffend. It is also evident that this model does not satisfy the predictive equality measurement when examining the false positive rates, since we can see in 2.e that Black defendants who do reoffend will only be predicted that they won't reoffend at a rate of 0.3181, whereas white defendants that do reoffend will be classifed as someone that won't reoffend at a rate of 0.5124. This means that since the false positive rates and false negative rates do not match for all races, it does not satisfy the predictive equality principle. 

Second, it also does not satisfy the statistical parity principle, since Black defendants are more likely to be predicted to reoffend than white defendants (based on 2.f, Black defendants predicted to reoffend rate being 0.5233, compared to white defendants predicted to reoffend rate being 0.3871). 

To check if the model satisfies demographic balance principle, we can write the following program:

In [11]:
actual_reoffend_all = testing_data.loc[testing_data["recidivism_outcome"] == 1]
actual_reoffend_black = testing_data.loc[(testing_data["Race = African American"] == 1) & (testing_data["recidivism_outcome"] == 1)]
actual_reoffend_white = testing_data.loc[(testing_data["Race = Caucasian"] == 1) & (testing_data["recidivism_outcome"] == 1)]

print("All defendants positive rate:", round(len(actual_reoffend_all) / len(testing_data), 4))
print("Black defendants positive rate:", round(len(actual_reoffend_black) / len(testing_data.loc[testing_data["Race = African American"] == 1]), 4))
print("White defendants positive rate:", round(len(actual_reoffend_white) / len(testing_data.loc[testing_data["Race = Caucasian"] == 1]),4))


All defendants positive rate: 0.4464
Black defendants positive rate: 0.5108
White defendants positive rate: 0.3871


This shows that the algorithm does not satisfy the demographic balance principle, since even though that the the predicted to reoffend rate for Black defendant roughly equals the recidivism rate for Black defendants, the predicted to reoffend rate  (30%) for white defendant is much lower than the actual recidivism rate (38.7%) for white defendants. Black defendants' predicted rate roughly being the same as the actual positice rate is not enough to call an algorithm fair for all demographics.

Based on these measurements and data, I would not trust this algorithm to be deployed on its own and with no human judgement, because it for sure violates two out of the the three core principles of algorithmic fairness (violates predictive equality and statistical parity), and we have not calculated calibration, which is a measurement that might be necessary to decide if an algorithm is fair. However, I do think that algorithms can help with aiding humans on their decision.

## Problem 3: Predicting Recidivism with Your Own Model (15 points)

Now you will train your own model to predict recidivism.

3a. Train a model of your choice. You can choose any input features that should be used as well as any pre-processing technique. You are welcome to use models which are not logistic regression models, but if you want to use logistic regression, that's also fine! (2 points)

I first started looking into a premade library in hopes that there's a premade model that accounts for sensitive data and weights different defendant's actions a bit differently to balance the outcomes out with regards to false positive rate and false negative rate. During my search, I found this premade model called fairlearn, and here were the results:

In [12]:
!conda install -c conda-forge fairlearn

# comment out previous line and run the following line if you are using pip
#!pip install fairlearn

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done


In [13]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference

In [14]:
# specify training/testing input/output features
train_input_features = training_data.drop(columns="recidivism_outcome")
train_output_feature = training_data["recidivism_outcome"]
test_input_features = testing_data.drop(columns="recidivism_outcome")
test_output_feature = testing_data["recidivism_outcome"]

#'Race = Other','Race = Asian','Race = Native American','Race = Caucasian','Race = Hispanic','Race = African American'
# select sensitive features - in this case I chose race to be the selective feature
selected_sensitive_features_train = train_input_features[['Race = Other','Race = Asian','Race = Native American','Race = Caucasian','Race = Hispanic','Race = African American']]
selected_sensitive_features_test = test_input_features[['Race = Other','Race = Asian','Race = Native American','Race = Caucasian','Race = Hispanic','Race = African American']]

# train a gradient boosting classifier on the training data:
model = GradientBoostingClassifier()
model.fit(train_input_features, train_output_feature)

# specify the constraints (what kind of fairness) we would like to achieve - in this case it'd demographic parity:
constraints = DemographicParity()
# ExponentiatedGradient is used to create a fair model by optimizing the base model's parameters to satisfy the fairness constraints.
fair_model = ExponentiatedGradient(model, constraints=constraints)
fair_model.fit(train_input_features, train_output_feature, sensitive_features=selected_sensitive_features_train)

# Output predictions and fairness metrics
predictions = fair_model.predict(test_input_features)
dp_difference = demographic_parity_difference(y_true=test_output_feature, y_pred=predictions, sensitive_features=selected_sensitive_features_test)
eod_difference = equalized_odds_difference(y_true=test_output_feature, y_pred=predictions, sensitive_features=selected_sensitive_features_test)


In [15]:
print(dp_difference)
print(eod_difference)

0.35714285714285715
0.5


In [16]:
def get_metrics(predictions, actual):
    accuracy = round((predictions == actual).mean(), 4)
    
    false_positives = ((predictions == 1) & (actual == 0)).sum()
    total_negatives = (actual == 0).sum()
    false_positive_rate = round(false_positives / total_negatives, 4)
    
    false_negatives = ((predictions == 0) & (actual == 1)).sum()
    total_positives = (actual == 1).sum()
    false_negative_rate = round(false_negatives / total_positives, 4)
    
    tp = ((predictions == 1) & (actual == 1)).sum()
    fp = ((predictions == 1) & (actual == 0)).sum()
    tn = ((predictions == 0) & (actual == 0)).sum()
    fn = ((predictions == 0) & (actual == 1)).sum()

    predicted_positive_rate = round((tp + fp) / (tp + fn + fp + tn), 4)
    
    return accuracy, false_positive_rate, false_negative_rate, predicted_positive_rate

def print_metrics(model, data, actual, label):
    predictions = model.predict(data.drop(columns=['recidivism_outcome']))
    accuracy, fpr, fnr, ppr = get_metrics(predictions, actual)
    
    print(f"{label} defendants metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"False Positive Rate: {fpr}")
    print(f"False Negative Rate: {fnr}")
    print(f"Predicted Positive Rate: {ppr}")
    print()

# Test performance
print_metrics(fair_model, testing_data, testing_data["recidivism_outcome"], "All")
print_metrics(fair_model, testing_data.loc[testing_data["Race = African American"] == 1], testing_data.loc[testing_data["Race = African American"] == 1]["recidivism_outcome"], "Black")
print_metrics(fair_model, testing_data.loc[testing_data["Race = Caucasian"] == 1], testing_data.loc[testing_data["Race = Caucasian"] == 1]["recidivism_outcome"], "White")
print_metrics(fair_model, testing_data.loc[testing_data["Race = Hispanic"] == 1], testing_data.loc[testing_data["Race = Hispanic"] == 1]["recidivism_outcome"], "Hispanic")
print_metrics(fair_model, testing_data.loc[testing_data["Race = Asian"] == 1], testing_data.loc[testing_data["Race = Asian"] == 1]["recidivism_outcome"], "Asian")
print_metrics(fair_model, testing_data.loc[testing_data["Race = Native American"] == 1], testing_data.loc[testing_data["Race = Native American"] == 1]["recidivism_outcome"], "Native American")
print_metrics(fair_model, testing_data.loc[testing_data["Race = Other"] == 1], testing_data.loc[testing_data["Race = Other"] == 1]["recidivism_outcome"], "Other")


All defendants metrics:
Accuracy: 0.689
False Positive Rate: 0.2304
False Negative Rate: 0.411
Predicted Positive Rate: 0.3905

Black defendants metrics:
Accuracy: 0.675
False Positive Rate: 0.2275
False Negative Rate: 0.4183
Predicted Positive Rate: 0.4084

White defendants metrics:
Accuracy: 0.7155
False Positive Rate: 0.2321
False Negative Rate: 0.3675
Predicted Positive Rate: 0.3871

Hispanic defendants metrics:
Accuracy: 0.7552
False Positive Rate: 0.2114
False Negative Rate: 0.3043
Predicted Positive Rate: 0.3854

Asian defendants metrics:
Accuracy: 0.4286
False Positive Rate: 0.6667
False Negative Rate: 0.0
Predicted Positive Rate: 0.7143

Native American defendants metrics:
Accuracy: 0.3333
False Positive Rate: 0.5
False Negative Rate: 0.75
Predicted Positive Rate: 0.3333

Other defendants metrics:
Accuracy: 0.7456
False Positive Rate: 0.2432
False Negative Rate: 0.275
Predicted Positive Rate: 0.4123



Initially, I got really excited, since the accuracy, false positive rate, false negative rate, predicted rates seemed to be much closer for Black and white defendants (about 5% difference compared to 13% initally for false positive rates, and similar differences for the rest), and it almost matched the average defendant's data as well, but that confused me why the Equalized Odds are so bad then, so I wanted to see how the model works on other demographics.

It turned out this model did worse on Asian and Native American defendants. Asian defendants had a similar accuracy rates, but a higher false positive rate (0.33) and a much lower (0.0!!!) false negative rate. Native American defendants, however had a lower accuracy rate and a 1.0 (!!!) false positive rate. This made me think that it's possible that there are not many Native American and/or Asian defendants in the dataset, so I wanted to see how many there are, since a difference like this (1.0 compared to 0.22) is shocking and would immediatley disqualify it from being a fair algorithm. To see this, I printed how many Asian and Native American defendants there are, along with the recidivism_outcome for each defendant:

In [17]:

def print_actual(model, data, actual, label):
    predictions = model.predict(data.drop(columns=['recidivism_outcome']))
    return predictions

predicted_asian = print_actual(fair_model, testing_data.loc[testing_data["Race = Asian"] == 1], testing_data.loc[testing_data["Race = Asian"] == 1]["recidivism_outcome"], "Asian")
predicted_native_american = print_actual(fair_model, testing_data.loc[testing_data["Race = Native American"] == 1], testing_data.loc[testing_data["Race = Native American"] == 1]["recidivism_outcome"], "Native American")

native_american_defendants_train = train_output_feature[train_input_features['Race = Native American'] == 1]
# print recidivism_outcome for Native American defendants:
print("Number of Native American defendant: " + str(len(native_american_defendants_train)))

print("Native American defendants' recidivism_outcome")
i  = 0
for outcome in native_american_defendants_train:
  print("Recidivism outcome: " + str(outcome) + " vs predicted: " + str(predicted_native_american[i]))
  i += 1

print()

asian_defendants_train = train_output_feature[train_input_features['Race = Asian'] == 1]
print("Number of Asian defendant: " + str(len(asian_defendants_train)))
print("Asian defendants' recidivism_outcome")
# print recidivism_outcome for Native American defendants:
j = 0
for outcome in asian_defendants_train:
  print("recidivism_outcome: " + str(outcome)+ " vs predicted: " + str(predicted_asian[j]))
  j += 1


Number of Native American defendant: 6
Native American defendants' recidivism_outcome
Recidivism outcome: 1 vs predicted: 1
Recidivism outcome: 0 vs predicted: 0
Recidivism outcome: 1 vs predicted: 0
Recidivism outcome: 1 vs predicted: 0
Recidivism outcome: 1 vs predicted: 1
Recidivism outcome: 0 vs predicted: 0

Number of Asian defendant: 7
Asian defendants' recidivism_outcome
recidivism_outcome: 0 vs predicted: 0
recidivism_outcome: 0 vs predicted: 1
recidivism_outcome: 1 vs predicted: 1
recidivism_outcome: 0 vs predicted: 0
recidivism_outcome: 0 vs predicted: 0
recidivism_outcome: 0 vs predicted: 0
recidivism_outcome: 0 vs predicted: 0


Seeing this I realized that the shocking prediction accuracy rates come from the problem that there is not enough training data on certain demographics. To fix this, I realized that maybe I should try to train the model based on Equalized Odds:

In [18]:
# train a gradient boosting classifier on the training data:
model_2 = GradientBoostingClassifier()
model_2.fit(train_input_features, train_output_feature)

# specify the constraints (what kind of fairness) we would like to achieve - in this case it'd demographic parity:
constraints_2 = EqualizedOdds()
# ExponentiatedGradient is used to create a fair model by optimizing the base model's parameters to satisfy the fairness constraints.
fair_model_2 = ExponentiatedGradient(model, constraints=constraints_2)
fair_model_2.fit(train_input_features, train_output_feature, sensitive_features=selected_sensitive_features_train)

# Output predictions and fairness metrics
predictions_2 = fair_model_2.predict(test_input_features)
dp_difference_2 = demographic_parity_difference(y_true=test_output_feature, y_pred=predictions, sensitive_features=selected_sensitive_features_test)
eod_difference_2 = equalized_odds_difference(y_true=test_output_feature, y_pred=predictions, sensitive_features=selected_sensitive_features_test)


In [19]:
# Test performance
print_metrics(fair_model_2, testing_data, testing_data["recidivism_outcome"], "All")
print_metrics(fair_model_2, testing_data.loc[testing_data["Race = African American"] == 1], testing_data.loc[testing_data["Race = African American"] == 1]["recidivism_outcome"], "Black")
print_metrics(fair_model_2, testing_data.loc[testing_data["Race = Caucasian"] == 1], testing_data.loc[testing_data["Race = Caucasian"] == 1]["recidivism_outcome"], "White")
print_metrics(fair_model_2, testing_data.loc[testing_data["Race = Hispanic"] == 1], testing_data.loc[testing_data["Race = Hispanic"] == 1]["recidivism_outcome"], "Hispanic")
print_metrics(fair_model_2, testing_data.loc[testing_data["Race = Asian"] == 1], testing_data.loc[testing_data["Race = Asian"] == 1]["recidivism_outcome"], "Asian")
print_metrics(fair_model_2, testing_data.loc[testing_data["Race = Native American"] == 1], testing_data.loc[testing_data["Race = Native American"] == 1]["recidivism_outcome"], "Native American")
print_metrics(fair_model_2, testing_data.loc[testing_data["Race = Other"] == 1], testing_data.loc[testing_data["Race = Other"] == 1]["recidivism_outcome"], "Other")


All defendants metrics:
Accuracy: 0.6909
False Positive Rate: 0.2429
False Negative Rate: 0.3913
Predicted Positive Rate: 0.4062

Black defendants metrics:
Accuracy: 0.6858
False Positive Rate: 0.2514
False Negative Rate: 0.3743
Predicted Positive Rate: 0.4425

White defendants metrics:
Accuracy: 0.7045
False Positive Rate: 0.2411
False Negative Rate: 0.3816
Predicted Positive Rate: 0.3871

Hispanic defendants metrics:
Accuracy: 0.6927
False Positive Rate: 0.2683
False Negative Rate: 0.3768
Predicted Positive Rate: 0.3958

Asian defendants metrics:
Accuracy: 0.8571
False Positive Rate: 0.1667
False Negative Rate: 0.0
Predicted Positive Rate: 0.2857

Native American defendants metrics:
Accuracy: 0.8333
False Positive Rate: 0.5
False Negative Rate: 0.0
Predicted Positive Rate: 0.8333

Other defendants metrics:
Accuracy: 0.6754
False Positive Rate: 0.2432
False Negative Rate: 0.475
Predicted Positive Rate: 0.3421



The issue of Native American and Asian defendants having highly different outcomes still stood, but since this likely stems from not having enough training data of these demographics, I thought it might be good to either modify the training data by either getting more Native American and Asian defendants, or use the average defendants (across all demographics) + their own demographics data to train it. However, since this homework focuses on Black vs white defendants, I decided to just exclude Native American and Asian for the data analysis, since there were just simply not enough defendants to get representative numbers on their outcomes. 

3b. Report the model's performance on (i) all, (ii) white, and (iii) Black defendants, using whatever metrics you believe are appropriate (at least 2). You are also welcome to evaluate performance on other sensitive/protected groups. (4 points)

Based on the previou model, the best results are from model 2, with the following scores:

In [20]:
# Test performance
print_metrics(fair_model_2, testing_data, testing_data["recidivism_outcome"], "All")
print_metrics(fair_model_2, testing_data.loc[testing_data["Race = African American"] == 1], testing_data.loc[testing_data["Race = African American"] == 1]["recidivism_outcome"], "Black")
print_metrics(fair_model_2, testing_data.loc[testing_data["Race = Caucasian"] == 1], testing_data.loc[testing_data["Race = Caucasian"] == 1]["recidivism_outcome"], "White")

All defendants metrics:
Accuracy: 0.6918
False Positive Rate: 0.2471
False Negative Rate: 0.3841
Predicted Positive Rate: 0.4117

Black defendants metrics:
Accuracy: 0.6795
False Positive Rate: 0.2606
False Negative Rate: 0.3779
Predicted Positive Rate: 0.4452

White defendants metrics:
Accuracy: 0.7073
False Positive Rate: 0.25
False Negative Rate: 0.3604
Predicted Positive Rate: 0.4008



Based on the false positive rates and false negative rates, it seems like this model does a fairly good job (or at least definitely better than the first model), making the equalized odds fairness test much better. The FPR is only about 1% more for Black defendants (compared to the 17% difference with the first model). The FNR has gone down to a 2% difference, with Black defendants being classified as falsely negative more than white defendants. The first model had a ~19% difference bewteen Black and white defendants' predictions, with white defendants much more likley to be classified as negative when they actually ended up reoffending. Therefore the equalized odds (otherwise known as Predictive equality) for this model is much better than previously. 

The statistical parity for this model is also better than previously, but still not perfect, since Black defendants still have a higher (44.6%) chance of being classified positive compared to white defendants (40%), but this is only about a 4.4% difference, while the first measure model classified Black defendants positive 52% of the time, while white defendants at only 30% of the time (which means that Black defendants were 1.7x more likely to be classified as positive).

This model, however, still does not satisfy the demographic balance principle, since since the predicted positive rate for Black defendants (44.6%) does not equal the recidivism rate for Black defendants (51%), and the predicted positive rate for white defendants (40.22%) does not equal the recidivism rate for white defendants (38.7%), although the predicted rate got much closer to the actual rate compared to the first model. 

To check if this model is well-calibrated, we can write the following code:

In [21]:
actual_reoffend_all = testing_data.loc[testing_data["recidivism_outcome"] == 1]
print("All defendants positive rate:", round(len(actual_reoffend_all) / len(testing_data), 4))


All defendants positive rate: 0.4464


In [22]:
import random

juvie_felony = ["1000", "0100", "0010", "0001"]
juvie_misdem = ["1000", "0100", "0010", "0001"]
juvie_other = ["1000", "0100", "0010", "0001"]
prior_conv = ["1000", "0100", "0010", "0001"]
degree = ["10", "01"]
disc = ["100000000000", "010000000000", "001000000000", "000100000000", "000010000000", "000001000000", "000000100000", "000000010000", "000000001000", "000000000100", "000000000010", "000000000001"]
age = ["100", "010", "001"]
gender =  ["10", "01"]
race = ["000100", "000001"]
outcome = ["0", "1"]

items = [juvie_felony, juvie_misdem, juvie_other, prior_conv, degree, disc, age, gender, race, outcome]

lst_of_random_test = []
for i in range(100):
  random_outcome_white = ""
  random_outcome_black = ""
  for item in items:
    if item != "race":
      new = str(random.choice(item))
      random_outcome_white += new
      random_outcome_black += new
      
    else:
      white = "000100"
      black = "000001"
      random_outcome_white += white
      random_outcome_black += black

  lst_of_random_test.append(random_outcome_white)
  lst_of_random_test.append(random_outcome_black)

def parse_string_to_list(input_string):
    return [int(char) for char in input_string]

result_list_test = []

for random_str in lst_of_random_test:
  result_list_test.append(parse_string_to_list(random_str))

outcome_lst_test = []
test_lst_test = []
for rnd in result_list_test:
   outcome_lst_test.append(rnd[:-1])
   test_lst_test.append(rnd[-1])



In [24]:
import csv

test = open('test.csv', 'w')
writer_test = csv.writer(test)


row = ["Juvenile felony count = 0","Juvenile felony count = 1","Juvenile felony count = 2","Juvenile felony count >= 3","Juvenile misdemeanor count = 0","Juvenile misdemeanor count = 1","Juvenile misdemeanor count = 2","Juvenile misdemeanor count >= 3","Juvenile other offense count = 0","Juvenile other offense count = 1","Juvenile other offense count = 2","Juvenile other offense count >= 3","Prior conviction count = 0","Prior conviction count = 1","Prior conviction count = 2","Prior conviction count >= 3","Charge degree = felony","Charge degree = misdemeanor","Charge description = no charge","Charge description = license issue","Charge description = public disturbance","Charge description = negligence","Charge description = drug related","Charge description = alcohol related","Charge description = weapons related","Charge description = evading arrest","Charge description = nonviolent harm","Charge description = theft/fraud/burglary","Charge description = lewdness/prostitution","Charge description = violent crime","Age < 25","Age >= 25 and <=45","Age > 45","Gender = Female","Gender = Male","Race = Other","Race = Asian","Race = Native American","Race = Caucasian","Race = Hispanic","Race = African American","recidivism_outcome"]
writer_test.writerow(row)

for row_i in result_list_test:
  writer_test.writerow(row_i)

test.close()

In [25]:
test_data_last = pd.read_csv('test.csv')

In [35]:

print_metrics(fair_model_2, test_data_last.loc[test_data_last["Race = African American"] == 1], test_data_last.loc[test_data_last["Race = African American"] == 1]["recidivism_outcome"], "Black")
print_metrics(fair_model_2, test_data_last.loc[test_data_last["Race = Caucasian"] == 1], test_data_last.loc[test_data_last["Race = Caucasian"] == 1]["recidivism_outcome"], "White")

Black defendants metrics:
Accuracy: 0.4914
False Positive Rate: 0.431
False Negative Rate: 0.5862
Predicted Positive Rate: 0.4224

White defendants metrics:
Accuracy: 0.5833
False Positive Rate: 0.5
False Negative Rate: 0.3158
Predicted Positive Rate: 0.5833



This model gives us the following output:

Black defendants metrics:
Accuracy: 0.4914
False Positive Rate: 0.431
False Negative Rate: 0.5862
Predicted Positive Rate: 0.4224

White defendants metrics:
Accuracy: 0.5833
False Positive Rate: 0.5
False Negative Rate: 0.3158
Predicted Positive Rate: 0.5833

Which shows us that this model is clearly not calibrated, since the model did not output the same results for the same inputs for Black and white defendants. 

3c. Write two paragraphs defending your model design choices, and explaining why you designed the model the way you did. Refer to results from 3b to provide evidence. You're welcome to write two paragraphs explaining why you don't think models should be used in criminal risk prediction at all - this is a reasonable perspective! - but you still need to provide quantitative or non-quantitative evidence to back up your claims. (9 points)

Designing this final model for criminal risk prediction, I decided to address issues of equalized odds and statistical parity. Although the model's output metrics for Black and white defendants, such as accuracy, false positive rate, false negative rate, and predicted positive rate seemed to show that it almost reached equalized odd and statistical parity, it clearly lacked  calibration. We know that it does not meet the requirement to be well calibrated, since the same inputs give different results for people in different demographic groups. I attempted various modifications, including excluding race, age, gender, and juvenile data from the training set, but these adjustments did not significantly improve the model's performance. Instead, removing these variables resulted in an accuracy barely surpassing random guessing, emphasizing the complexity of achieving fairness without compromising predictive accuracy.

Even after trying to get my model to satisfy the equalized odds by using a built in library made by developers specifically for this method, I would not trust it to be used criminal risk prediciton, since some demographic groups are severly underrepresented (Indigenous and Asian defendants) so their results were heavily biased against them. This poses ethical concerns, as excluding certain demographics from the model undermines its applicability and fairness.

Additionally, in class we discussed that calibration, or equalized odds alone is insufficient for a fair algorithm. While calibration ensures consistent results for the same input across demographic groups, achieving equalized odds becomes challenging without either perfect prediction or calibration, and vice versa, as highlighted in studies by Kleinberg, Mullainathan, Raghavan and Chouldechova (2016). The limitations of the model mean that there are too many challenges and ethical considerations to use them in high-stakes situations like criminal risk prediction.

# Sources cited

Please cite any sources used to complete this homework in the markdown cell below. Nobody remembers everything, and it's always a good idea to use documentation and online resources to ensure you're growing your skills.

Note that copying text from generative AI technology, such as ChatGPT, will be considered plagiarism (and hence result in a 0 grade on this submission). You are allowed to use ChatGPT as a general educational resource (the same way you would a webpage, without copying from it). But if you used ChatGPT as an educational resource, you must include (a) your prompts, (b) ChatGPT's responses, and (c) your validation of why ChatGPT's response is correct; simply noting that the code makes sense does not suffice as an explanation.

**SOURCES USED:**

Reading in a CSV file: https://stackoverflow.com/questions/58185487/how-to-read-csv-file-into-jupyter-notebook#61956061


Selecting rows with certain values: https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values

Selecting columns for training: https://stackoverflow.com/questions/20230326/retrieve-dataframe-of-all-but-one-specified-column


Confusion matrix: https://blog.nillsf.com/index.php/2020/05/23/confusion-matrix-accuracy-recall-precision-false-positive-rate-and-f-scores-explained/

Fairlearn documentation: https://fairlearn.org/v0.10/user_guide/fairness_in_machine_learning.html#group-fairness-sensitive-features

Write to CSV file: https://www.pythontutorial.net/python-basics/python-write-csv-file/